In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs 
import os
import json
from tqdm import tqdm
import fuckit

In [62]:
def parse_outlet(soup):
    json_data = soup.find('script', {'type': 'application/ld+json'}).text
    json_data = json.loads(json_data)
    item = {}
    item['source'] = 'thuisbezorgd'
    item['id_source'] = item['url'] = json_data['@id']
    item['category'] = json_data['@type']
    item['name'] = json_data['name']
    item['image_url'] = json_data['image']
    item['street'] = json_data['address']['streetAddress']
    item['country'] = json_data['address']['addressCountry']
    item['postal_code'] = json_data['address']['postalCode']
    item['city'] = json_data['address']['addressLocality']
    item['address'] = item['street'] + ', ' + item['city'] + ', ' + item['postal_code']
    with fuckit:
        item['rating'] = json_data['aggregateRating']['ratingValue']
        item['reviews_nr'] = json_data['aggregateRating']['reviewCount']
    item['lat'] = json_data['geo']['latitude']
    item['lon'] = json_data['geo']['longitude']
    item['currency'] = json_data['potentialAction']['priceSpecification']['priceCurrency']
    with fuckit:
        item['opening_hour'] = '; '.join(json_data['openingHours'])
    with fuckit:
        item['description'] = json_data['description']
    return item

In [84]:
filenames = os.listdir('BE_202104/html_cache/')
items = []
for filename in tqdm(filenames):
    with open(f'BE_202104/html_cache/{filename}', 'r') as f:
        soup = bs(f.read(), 'html.parser')
    try:
        item = parse_outlet(soup)
    except:
        print (filename)
    next_data = soup.find('script', {'id': '__NEXT_DATA__'})
    if next_data is None:
        print (item['url'])
    items.append(item)
outlets = pd.DataFrame(items)
outlets

 93%|█████████▎| 79/85 [00:11<00:00,  7.07it/s]dominos-pizza-genk-europalaan.html
https://www.takeaway.com/be-en/menu/tonton-chami-centre
100%|██████████| 85/85 [00:12<00:00,  6.59it/s]


,source,id_source,url,category,name,image_url,street,country,postal_code,city,address,rating,reviews_nr,lat,lon,currency,opening_hour,description
0,thuisbezorgd,https://www.takeaway.com/be-en/menu/wok-classic,https://www.takeaway.com/be-en/menu/wok-classic,Restaurant,Wok Classic,https://static.takeaway.com/images/restaurants...,Rue des Echevins 1,Belgium,5070,Fosses-la-ville,"Rue des Echevins 1, Fosses-la-ville, 5070",5,273,50.3953454,4.6977415,EUR,Su 12:00-14:30; Su-Th 17:00-21:00; Fr-Sa 17:00...,
1,thuisbezorgd,https://www.takeaway.com/be-en/menu/la-petite-...,https://www.takeaway.com/be-en/menu/la-petite-...,Restaurant,La Petite Maison,https://static.takeaway.com/images/restaurants...,place de la Digue 33,Belgium,6000,Charleroi,"place de la Digue 33, Charleroi, 6000",5,876,50.4097233,4.4384387,EUR,Su-Mo 11:15-22:30; We 11:15-22:30; Th 12:00-22...,
2,thuisbezorgd,https://www.takeaway.com/be-en/menu/la-copita,https://www.takeaway.com/be-en/menu/la-copita,Restaurant,La Copita,https://static.takeaway.com/images/restaurants...,Heideplaats 43,Belgium,2845,Niel,"Heideplaats 43, Niel, 2845",4.5,592,51.1075272,4.3269128,EUR,Su 15:45-20:45; Fr 17:00-21:30; Sa 16:00-21:30,"Beste klanten,\n\nDANK VOOR JULLIE MASSAAL AAN..."
3,thuisbezorgd,https://www.takeaway.com/be-en/menu/dam-menu,https://www.takeaway.com/be-en/menu/dam-menu,Restaurant,Dam Menu,https://static.takeaway.com/images/restaurants...,Nederkouter 139,Belgium,9000,Gent,"Nederkouter 139, Gent, 9000",0,0,51.0464889,3.7222181,EUR,Tu-Sa 11:00-18:00,
4,thuisbezorgd,https://www.takeaway.com/be-en/menu/valide-bur...,https://www.takeaway.com/be-en/menu/valide-bur...,Restaurant,Validé Burgers & Sandwichs,https://static.takeaway.com/images/restaurants...,Rue Auguste Buisseret 51,Belgium,4000,Liege,"Rue Auguste Buisseret 51, Liege, 4000",3.5,86,50.6217842,5.5727049,EUR,Su 16:15-21:40; Mo-Sa 11:00-13:50; Mo-Sa 16:30...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,thuisbezorgd,https://www.takeaway.com/be-en/menu/king-of-kebap,https://www.takeaway.com/be-en/menu/king-of-kebap,Restaurant,King of Kebab,https://static.takeaway.com/images/restaurants...,Grand place 35,Belgium,7800,Ath,"Grand place 35, Ath, 7800",4.5,91,50.6312344,3.7774856,EUR,Su-Mo 12:00-23:30; We 12:00-23:30; Th 12:00-23...,
81,thuisbezorgd,https://www.takeaway.com/be-en/menu/mcdonalds-...,https://www.takeaway.com/be-en/menu/mcdonalds-...,Restaurant,McDonald's - Grand Place,https://static.takeaway.com/images/restaurants...,Grand Place 39,Belgium,7000,Mons,"Grand Place 39, Mons, 7000",4.5,100,50.4537972,3.9523362,EUR,Su-Sa 11:00-21:00,
82,thuisbezorgd,https://www.takeaway.com/be-en/menu/giraffe-wi...,https://www.takeaway.com/be-en/menu/giraffe-wi...,Restaurant,Frituur Giraffe,https://static.takeaway.com/images/restaurants...,Leopold Decouxlaan 91,Belgium,3012,Wilsele,"Leopold Decouxlaan 91, Wilsele, 3012",3.5,42,50.8940146,4.6969533,EUR,Su 17:30-21:00; Tu 17:30-21:00; We 17:30-21:00...,
83,thuisbezorgd,https://www.takeaway.com/be-en/menu/regina-bru...,https://www.takeaway.com/be-en/menu/regina-bru...,Restaurant,Regina,https://static.takeaway.com/images/restaurants...,Chaussée de Roodebeek 205,Belgium,1200,Woluwé-Saint-Lambert,"Chaussée de Roodebeek 205, Woluwé-Saint-Lamber...",3.5,14,50.8481989,4.4282621,EUR,Mo-Fr 12:00-14:15; Mo-Sa 17:45-21:15,


In [66]:
outlets['country'].value_counts()

Belgium    78
BE          3
Name: country, dtype: int64